# Test for the Modified Number Operator
## Construct the Hamiltonians for 1d lattice

In [4]:
import sys
sys.path.append("../")
import fh_comm as fhc
import numpy as np

In [5]:
v = -1
u = 1
translatt = fhc.SubLattice(np.array([[2]]))
h0 = fhc.SumOp([fhc.HoppingOp(( 0,), ( 1,), s, v) for s in [0, 1]])
h1 = fhc.SumOp([fhc.HoppingOp((-1,), ( 0,), s, v) for s in [0, 1]])
h2 = fhc.SumOp([fhc.ProductOp([fhc.ModifiedNumOp((x,), s, 1) for s in [0, 1]], u) for x in [0, 1]])
h2_num = [[fhc.NumberOp((x,),s,u) for x in [0,1]] for s in [0,1]]

In [6]:
# evaluate bound for 3 Hamiltonian terms
comm_bound_terms = fhc.commutator_bound_strang(3)
for cbt in comm_bound_terms:
    print(cbt)

1/24 * [H_0, [H_1, H_0]]
1/12 * [H_1, [H_1, H_0]]
1/12 * [H_2, [H_1, H_0]]
1/24 * [H_0, [H_2, H_0]]
1/12 * [H_1, [H_2, H_0]]
1/12 * [H_2, [H_2, H_0]]
1/24 * [H_1, [H_2, H_1]]
1/12 * [H_2, [H_2, H_1]]


Notice: Because we just modified H_2, than we just need to compute the term with H_2,
s.t. [H_2, [H_1, H_0]], [H_0, [H_2, H_0]], [H_1, [H_2, H_0]], [H_2, [H_2, H_0]], [H_1, [H_2, H_1]], [H_2, [H_2, H_1]]

In [ ]:
# results of the first level commutator
result10 = fhc.commutator(h1,h0)
result20 = fhc.commutator(h2,h0)
result21 = fhc.commutator(h2,h1)

In [ ]:
# results of the 6 terms with H_2
result1 = fhc.commutator(h2,result10)
result2 = fhc.commutator(h0,result20)
result3 = fhc.commutator(h1,result20)
result4 = fhc.commutator(h2,result20)
result5 = fhc.commutator(h1,result21)
result6 = fhc.commutator(h2,result21)

In [ ]:
# we compute the results by hand, and the 